In [4]:
%pip install crewai langchain langchain-openai --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install python-dotenv --quiet
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [6]:
import subprocess

result = subprocess.run(["ls", "-l"])
print(result.stdout)
print(result.returncode)


total 288
-rw-r--r--  1 patrickdebois  staff   3678 Jun 14 14:17 1_simple_llm_langchain.ipynb
-rw-r--r--  1 patrickdebois  staff  43638 Jun 14 14:35 2_react_prompt_langchain.ipynb
-rw-r--r--  1 patrickdebois  staff  18488 Jun 14 14:34 3_dev_solo_crewai.ipynb
-rw-r--r--  1 patrickdebois  staff  11402 Jun 14 14:36 4_dev_ops_crewai.ipynb
-rw-r--r--  1 patrickdebois  staff   1867 Jun 13 17:34 5_dev_sec_ops_langgraph_crewai.ipynb
-rw-r--r--  1 patrickdebois  staff    811 Jun 13 17:47 README.md
drwxr-xr-x  3 patrickdebois  staff     96 Jun 14 14:17 __pycache__
-rw-r--r--  1 patrickdebois  staff  48884 Jun 13 14:37 poetry.lock
-rw-r--r--  1 patrickdebois  staff   4058 Jun 14 14:07 pretty_print_callback_handler.py
-rw-r--r--  1 patrickdebois  staff    340 Jun 13 14:37 pyproject.toml
None
0


In [7]:
from crewai import Agent, Task, Crew, Process


In [8]:
# Prepare the LLM for the agent to use
from pretty_print_callback_handler import PrettyPrintCallbackHandler
pretty_callback = PrettyPrintCallbackHandler()

from langchain_openai import OpenAI
agent_llm = OpenAI(temperature=0, callbacks=[pretty_callback])


In [9]:
developer = Agent(
  role='Senior Developer',
  goal='Develop code that is high qualuty, scalable, and maintainable.',
  backstory="""I'm a senior developer with 10 years of experience. I have worked on a variety of projects and have a deep understanding of software development best practices. 
  I am passionate about writing clean, efficient code and am always looking for ways to improve my skills.
  """,
  verbose=True,
  llm=agent_llm
  # You can pass an optional llm attribute specifying what model you wanna use.
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
)


In [10]:
ops = Agent(
  role='Senior DevOps Engineer',
  goal='Make sure the code works repeatably and is idempotent.',
  backstory="""I'm a senior Devops Engineer with 10 years of experience.
  I've written a lot of bash scripts as I live in the terminal all day.`
  """,
  verbose=True,
  llm=agent_llm
  # You can pass an optional llm attribute specifying what model you wanna use.
  # llm=ChatOpenAI(model_name="gpt-3.5", temperature=0.7),
)


In [11]:

# Create tasks for your agents
task1 = Task(
  description="""Generate me shell script that will list all files recursively in the current directory.""",
  expected_output="A script shown as markdown code block",
  agent=developer,
  llm=agent_llm
)

In [12]:
# Create tasks for your agents
task2 = Task(
  description="""Analyze a shell script and make sure it's compatible with most of the Linux systems.""",
  expected_output="A script shown as markdown code block",
  agent=ops,
  llm=agent_llm
)

In [13]:

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[developer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)

 [DEBUG]: == Working Agent: Senior Developer
 [INFO]: == Starting Task: Generate me shell script that will list all files recursively in the current directory.


> Entering new CrewAgentExecutor chain...

[llm][start] - prompts: You are Senior Developer. I'm a senior developer with 10 years of experience. I have worked on a variety of projects and have a deep understanding of software development best practices. 
  I am passionate about writing clean, efficient code and am always looking for ways to improve my skills.
  
Your personal goal is: Develop code that is high qualuty, scalable, and maintainable.To give my best complete final answer to the task use the exact following format:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.
Your final answer must be the great and the most complete as possible, it must be outcome described.

I MUST use these formats, my job depends on it!
Current Task: Generate me shell script that will list all f